## Libraries

In [1]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [2]:
if IN_COLAB:
    !pip install transformers
    !pip install datasets
    !pip install evaluate
    !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [3]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA'
else:
    root_path = '/Users/kscaria/Library/CloudStorage/GoogleDrive-scariakevin1@gmail.com/My Drive/Knowledge/MSIT/Research/InstructABSA'
    
use_mps = True if torch.has_mps else False
os.chdir(root_path)

In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.utils import T5ATSC
from InstructABSA.data_prep import ModelReadyData

In [5]:
rest_train_file_path = './Data/semeval14/ABSA_TrainData/Restaurants_Train_v2.csv'
laptops_train_file_path = './Data/semeval14/ABSA_TrainData/Laptop_Train_v2.csv'
rest_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Restaurants_Test_Gold.csv'
laptops_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Laptops_Test_Gold.csv'

# Load the data
res_tr_df = pd.read_csv(rest_train_file_path)
lap_tr_df = pd.read_csv(laptops_train_file_path)
res_te_df = pd.read_csv(rest_test_file_path)
lap_te_df = pd.read_csv(laptops_test_file_path)

In [6]:
# Extract the aspect term and polarity for each review
model_data = ModelReadyData()
res_tr_df = model_data.extract_rowwise_aspect_polarity(res_tr_df, on='aspectTerms', by="term", min_val=1)
lap_tr_df = model_data.extract_rowwise_aspect_polarity(lap_tr_df, on='aspectTerms', by="term", min_val=1)
res_te_df = model_data.extract_rowwise_aspect_polarity(res_te_df, on='aspectTerms', by="term", min_val=1)
lap_te_df = model_data.extract_rowwise_aspect_polarity(lap_te_df, on='aspectTerms', by="term", min_val=1)

# Get the input text into the required format
bos_instruction_lap = """Definition: The output will be 'positive' if the aspect identified in the sentence contains a positive sentiment. If the sentiment of the identified aspect in the input is negative the answer will be 'negative'. 
Otherwise, the output should be 'neutral'. For aspects which are classified as noaspectterm, the sentiment is none. 
Positive example 1- input: I charge it at night and skip taking the cord with me because of the good battery life. Aspect: battery life. output: positive. 
Positive example 2- input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!. Aspect: Photobooth. output: positive. 
Now complete the following example-
input: """

bos_instruction_res = """Definition: The output will be 'positive' if the aspect identified in the sentence contains a positive sentiment. If the sentiment of the identified aspect in the input is negative the answer will be 'negative'. 
Otherwise, the output should be 'neutral'. For aspects which are classified as noaspectterm, the sentiment is none. 
Positive example 1- input: With the great variety on the menu , I eat here often and never get bored. Aspect: menu. output: positive. 
Positive example 2- input: Great food, good size menu, great service and an unpretensious setting. Aspect: food. output: positive. 
Now complete the following example-
input: """
delim_instruction = ' Aspect: '
eos_insrtuction = '. Output:'

res_tr_df = model_data.create_data_in_atsc_format(res_tr_df, 'raw_text', 'aspect', 'labels', bos_instruction_res, delim_instruction, eos_insrtuction)
lap_tr_df = model_data.create_data_in_atsc_format(lap_tr_df, 'raw_text', 'aspect', 'labels', bos_instruction_lap, delim_instruction, eos_insrtuction)
res_te_df = model_data.create_data_in_atsc_format(res_te_df, 'raw_text', 'aspect', 'labels', bos_instruction_res, delim_instruction, eos_insrtuction)
lap_te_df = model_data.create_data_in_atsc_format(lap_te_df, 'raw_text', 'aspect', 'labels', bos_instruction_lap, delim_instruction, eos_insrtuction)

In [7]:
res_tr_df['labels'].value_counts()

positive    2164
none        1020
negative     805
neutral      633
conflict      91
Name: labels, dtype: int64

In [8]:
res_te_df['labels'].value_counts()

positive    728
neutral     196
negative    196
none        194
conflict     14
Name: labels, dtype: int64

In [ ]:
lap_tr_df['labels'].value_counts()

none        1557
positive     987
negative     866
neutral      460
conflict      45
Name: labels, dtype: int64

In [ ]:
lap_te_df['labels'].value_counts()

none        378
positive    341
neutral     169
negative    128
conflict     16
Name: labels, dtype: int64

In [ ]:
# Experimentation
experiment_id = 'laptops'
experiment_name = 'laptops_instruct'
model_checkpoint = 'allenai/tk-instruct-base-def-pos'

# Create T5 utils object
t5_exp = T5ATSC(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATSC')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
id_dataset, ood_dataset, id_tokenized_dataset, ood_tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'use_mps_device':use_mps
}

In [ ]:
# Train model
model_trainer = t5_exp.train(id_tokenized_dataset, **training_args)

***** Running training *****
  Num examples = 6697
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1676
  Number of trainable parameters = 247534848


Trainer device: cuda:0

Model training started ....


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.231814
2,0.368400,0.275331
3,0.159200,0.298369
4,0.107900,0.361637


***** Running Evaluation *****
  Num examples = 1716
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1716
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1716
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1716
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-laptops_instruct/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-laptops_instruct/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-laptops_instruct/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-lapto

In [ ]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

try:
    model_trainer
except:
    model_trainer = None

# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = id_tokenized_dataset, sample_set = 'train', trained_model_path = model_out_path)

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = id_tokenized_dataset, sample_set = 'validation', trained_model_path = model_out_path)

# Get prediction labels - OOD Training set
ood_tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = ood_tokenized_dataset, sample_set = 'train', trained_model_path = model_out_path)

# Get prediction labels - OOD Testing set
ood_te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = ood_tokenized_dataset, sample_set = 'validation', trained_model_path = model_out_path)

***** Running Prediction *****
  Num examples = 3915
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-laptops_instruct/checkpoints


***** Running Prediction *****
  Num examples = 1032
  Batch size = 16


***** Running Prediction *****
  Num examples = 4713
  Batch size = 16


***** Running Prediction *****
  Num examples = 1328
  Batch size = 16


In [ ]:
# Add new column in the respective dataframes
id_tr_df = pd.DataFrame(id_dataset['train'])
id_te_df = pd.DataFrame(id_dataset['validation'])
ood_tr_df = pd.DataFrame(ood_dataset['train'])
ood_te_df = pd.DataFrame(ood_dataset['validation'])

id_tr_df['pred_labels'] = id_tr_pred_labels
id_te_df['pred_labels'] = id_te_pred_labels
ood_tr_df['pred_labels'] = ood_tr_pred_labels
ood_te_df['pred_labels'] = ood_te_pred_labels

# Metrics
print('In domain train accuracy: ', id_tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(id_tr_df))
print('In domain test accuracy: ', id_te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(id_te_df))

ood_df = pd.concat([ood_tr_df, ood_te_df])
print('Out of domain accuracy: ', ood_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(ood_df))
print('Out of domain train accuracy: ', ood_tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(ood_tr_df))
print('Out of domain test accuracy: ', ood_te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(ood_te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
id_tr_df.to_csv(os.path.join(dump_path, id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
id_te_df.to_csv(os.path.join(dump_path, id_te_filename), index = False)

ood_tr_filename = t5_exp.get_csv_filename(experiment_id)[2]
ood_tr_df.to_csv(os.path.join(dump_path, ood_tr_filename), index = False)

ood_te_filename = t5_exp.get_csv_filename(experiment_id)[3]
ood_te_df.to_csv(os.path.join(dump_path, ood_te_filename), index = False)

In domain train accuracy:  96.32183908045977
In domain test accuracy:  88.37209302325581
Out of domain accuracy:  81.4434696242344
Out of domain train accuracy:  80.18247400806281
Out of domain test accuracy:  85.91867469879519
